In [1]:
import os
import pandas as pd
import io

In [2]:
def writeEntry(text, data, f, quotes):
    '''
    If the data is not missing write the property to the provided file.

    @text   string of the property
    @data   data representing the property
    @f      file to which to write the property in .mcf format
    @quotes boolean specifying if quotes need to be added around the data entry 
          for a string
    '''
    if len(data) > 0  and data!=',':
        if quotes:
            f.write(text + '"' + data + '"\n')
    else:
        f.write(text + data + '\n')

def addToDict(dictionary, key, value):
    '''
    Add value to a dictionary if the value is not empty.
    @dictionary dict to which to add the key, value pair
    @key        key to which to associate the value
    @value      value to be associated with the key
    @return     updated dictionary
    '''
    if len(value) > 0:
        dictionary[key] = value
    return(dictionary)

def writeEntry2(dictionary, key, f, quotes):
    '''
    If the data is not missing write the property to the provided file.

    @dictionary   stores key (property), value as a pair
    @key          string of the property and the key in the dictionary
    @f            file to which to write the property in .mcf format
    @quotes       boolean specifying if quotes need to be added around the data
                entry for a string
    '''
    if key in dictionary and quotes:
        f.write(key + ': "' + dictionary[key] + '"\n')
    elif key in dictionary:
        f.write(key + ': ' + dictionary[key] + '\n')

def writeEntry3(prop, value, f, quotes):
    '''
    If the data is not missing write the property to the provided file.

    @prop     string of the property to write to the file
    @value    string of the value to write to the file
    @f        file to which to write the property in .mcf format
    @quotes   boolean specifying if quotes need to be added around the data
              entry for a string
  '''
    if len(value) > 0 and quotes and value!=',':
        f.write(prop + ': "'+ value + '"\n')
    elif len(value) > 0  and value!=',':
        f.write(prop + value + '\n')
        
def writeList(dictionary, value, f):
    '''
    If the data is not missing write the property to the provided file.
    Properties will be a list of values that are comma seperated. It
    assumes the list is of dcids to be written to a property.

    @dictionary   stores key (property), value as a pair
    @value         string of the property and the key in the dictionary
    @f            file to which to write the property in .mcf format
    '''
    if value in dictionary:
        w.write(value + ': dcid:' + dictionary[value][0])
        if len(dictionary[value]) > 0:
            for ID in dictionary[value][1:]:
                w.write(',dcid:' + ID)
        w.write('\n')
        
def writeList2(dictionary, value, f):
    '''
    If the data is not missing write the property to the provided file.
    Properties will be a list of values that are comma seperated. It does
    not assume that the list are dcid's to be written to a property as a
    list.

    @dictionary   stores key (property), value as a pair
    @value         string of the property and the key in the dictionary
    @f            file to which to write the property in .mcf format
    '''
    if value in dictionary:
        w.write(value + ': ' + dictionary[value][0])
        if len(dictionary[value]) > 0:
            for ID in dictionary[value][1:]:
                w.write(',' + ID)
        w.write('\n')
        
def convertToStrandEnum(strand):
    if strand == '+':
        return('dcid:StrandOrientationPositive')
    if strand == '-':
        return('dcid:StrandOrientationNegative')
    
def writeExonCoordinates(starts, stops, f):
    index = 0
    if len(starts) > 0:
        f.write('exonCoordinates: [' + starts[0] + ' ' + stops[0] + ' Position]')
        index += 1
    while index < len(starts):
        f.write(',[' + starts[index] + ' ' + stops[index] + ' Position]')
        index += 1
    f.write('\n')

def writeNomenclatureStatus(ns, f):
    # convert the nomenclature status to the dcid of the corresponding enum
    nomenclatureStatus = ''
    if ns == 'O':
        nomenclatureStatus = 'dcid:NomenclatureStatusOfficial'
    elif ns == 'I':
        nomenclatureStatus = 'dcid:NomenclatureStatusInterim'
    elif line[12] == '-':
        ns = 'dcid:NomenclatureStatusNCBIsupplied'
    writeEntry3('nomenclatureStatus: ', nomenclatureStatus, f, False)
    
def writeTypeOfGene(gt, f):
    geneType = ''
    if gt == 'biological-region':
        geneType = 'dcid:TypeOfGeneBiologicalRegion'
    elif gt == 'ncRNA':
        geneType = 'dcid:TypeOfGenencRNA'
    elif gt == 'other':
        geneType = 'dcid:TypeOfGeneOther'
    elif gt == 'protein-coding':
        geneType = 'dcid:TypeOfGeneProteinCoding'
    elif gt == 'pseudo':
        geneType = 'dcid:TypeOfGenePseudo'
    elif gt == 'rRNA':
        geneType = 'dcid:TypeOfGenerRNA'
    elif gt == 'scRNA':
        geneType = 'dcid:TypeOfGenescRNA'
    elif gt == 'snRNA':
        geneType = 'dcid:TypeOfGenesnRNA'
    elif gt == 'snoRNA':
        geneType = 'dcid:TypeOfGenesnoRNA'
    elif gt == 'tRNA':
        geneType = 'dcid:TypeOfGenetRNA'
    elif gt == 'miscRNA':
        geneType = 'dcid:TypeOfGenemiscRNA'
    elif gt == 'unknown':
        geneType = 'dcid:TypeOfGeneUnknown'
    else:
        print(gt)
    writeEntry3('typeOfGene: ', geneType, f, False)

In [3]:
# set assembly name
assembly = 'hg38'

# set species name
species = 'Homo_sapiens'
species_abv = 'hs'

# set file path
path = '/Users/spiekos/Documents/files+mcf_creation_code/gene_coordinates'

# save info from RefSeq file for Genes
dict_gene = {}

# write the RefSeq instances to the *_RefSeq.mcf output file
refSeq = assembly + '_gene_coordinates.txt'
refSeq_file = os.path.join(path, refSeq)
r = open(refSeq_file, 'r')
transcript_output_file = assembly + '_RNATranscript.mcf'
refSeq_mcf_file = os.path.join(path, 'RNATranscript_Gene_mcfs', transcript_output_file)
w = open(refSeq_mcf_file, 'w')
next(r)
count = 0
for line in r:
    line = line.strip('\r\n').strip('\\').split('\t')
    count += 1
    dcid = 'bio/'  + assembly + '_' + line[0].replace(',', '')
    w.write('Node: dcid:' + dcid + '\n')
    w.write('ofSpecies: dcid:bio/' + species_abv + '\n')
    w.write('name: "' + line[0].replace(',', '') + '"\n')
    w.write('inGenomeAssembly: dcid:bio/' + assembly + '\n')
    w.write('strandOrientation: '+ convertToStrandEnum(line[2].replace(',', ''))  + '\n')
    w.write('inChromosome: dcid:bio/' + assembly + '_' + line[1].replace(',', '')  + '\n')
    w.write('transcriptionCoordinates: [' + line[3].replace(',', '')  + ' ' + line[4].replace(',', '')  + ' Position]\n')
    w.write('codingCoordinates: [' + line[5].replace(',', '')  + ' ' + line[6].replace(',', '')  + ' Position]\n')
    writeEntry('exonFrame: ', line[7].replace(',', '') , w, False)
    writeExonCoordinates(line[8].split(',')[:-1], line[9].split(',')[:-1], w)
    if species_abv == 'hs':
        writeEntry('makesProtein: ', 'dcid:bio/UniProt_' + line[10].replace(',', '') + '_HUMAN', w, True)
    else:
        writeEntry('makesProtein: ', line[10].replace(',', '') , w, True)
    writeEntry('geneSymbol: ', line[12].replace(',', '') , w, True)
    writeEntry('refSeqID: ', line[13].replace(',', '') , w, True)
    w.write('typeOf: schema:RNATranscript\n')
    w.write('\n')
    gene_name = line[12].replace(',', '')
    if gene_name not in dict_gene.keys():
        dict_gene[gene_name] = {'strandOrientation': line[2].replace(',', '')}
        dict_gene[gene_name]['hasRNATranscript'] = [dcid]
        dict_gene[gene_name]['inChromosome'] = line[1].replace(',', '')
    else:
        dict_gene[gene_name]['hasRNATranscript'].append(dcid)
    if len(line[13]) > 0 and line[13] != ',':
        refSeq = line[13].strip(',')
        if 'refSeqID' not in dict_gene[gene_name].keys():
            dict_gene[gene_name]['refSeqID'] = [refSeq]
        elif line[13] not in dict_gene[gene_name]['refSeqID']:
            dict_gene[gene_name]['refSeqID'].append(refSeq)
r.close()
w.close()
print('Number of UCSCTranscript instances: ' + str(count))

Number of UCSCTranscript instances: 226811


In [4]:
# keep track of recorded gene - if duplicated entries, take the largest region 
dict_recorded_genes = {}

# write the UCSC gene file to the dict_gene
gene_file = assembly + '_known_canonical_gene_coordinates.txt'
UCSC_gene_file = os.path.join(path, gene_file)
r = open(UCSC_gene_file, 'r')
next(r)
for line in r:
    line = line.strip('\r\n').strip('\\').split('\t')
    gene_size = int(line[2]) - int(line[1])
    gene_name = line[5].replace(',', '')
    if gene_name not in dict_recorded_genes:
        dict_recorded_genes[gene_name] = gene_size
    if gene_size < dict_recorded_genes[gene_name]:
        continue
    if gene_name not in dict_gene:
        dict_gene[gene_name] = {}
    dict_recorded_genes[gene_name] = gene_size
    addToDict(dict_gene[gene_name], 'inChromosome', line[0])
    addToDict(dict_gene[gene_name], 'cannonicalStart', line[1])
    addToDict(dict_gene[gene_name], 'cannonicalStop', line[2])
    addToDict(dict_gene[gene_name], 'mRNA', line[4].replace(',', ''))
    addToDict(dict_gene[gene_name], 'geneSymbol', line[5].replace(',', ''))
    addToDict(dict_gene[gene_name], 'ncbiProteinAccessionNumber', line[7].replace(',', ''))
    addToDict(dict_gene[gene_name], 'description', line[8])
    if len(line[6]) > 1:
        if 'refSeqID' not in dict_gene[gene_name].keys():
            dict_gene[gene_name]['refSeqID'] = [line[6].replace(',', '')]
        elif line[6] not in dict_gene[gene_name]['refSeqID']:
            dict_gene[gene_name]['refSeqID'].append(line[6].replace(',', ''))
r.close()

In [5]:
# write the gene nodes to a file in mcf format
gene_output_file = assembly + '_gene.mcf'
gene_mcf_file = os.path.join(path, 'RNATranscript_Gene_mcfs', gene_output_file)
w = open(gene_mcf_file, 'w')
count = 0
for key in dict_gene:
    count += 1
    dcid = 'bio/' + assembly + '_' + key
    w.write('Node: dcid:' + dcid + '\n')
    w.write('name: "' + key + '"\n')
    w.write('ofSpecies: dcid:bio/' + species_abv + '\n')
    w.write('inGenomeAssembly: dcid:bio/' + assembly + '\n')
    w.write('inChromosome: dcid:bio/' + assembly + '_' + dict_gene[key]['inChromosome'] + '\n')
    if 'cannonicalStart' in dict_gene[key] and 'cannonicalStop' in dict_gene[key]:
        w.write('genomicCoordinates: [' + dict_gene[key]['cannonicalStart'] + ' ' + \
                dict_gene[key]['cannonicalStop'] + ' Position]\n')
    if 'strandOrientation' in dict_gene[key]:
        w.write('strandOrientation: '+ convertToStrandEnum(dict_gene[key]['strandOrientation']) + '\n')
    w.write('geneSymbol: "' + key + '"\n')
    writeEntry2(dict_gene[key], 'mRNA', w, True)
    writeList2(dict_gene[key], 'refSeqID', w)
    writeList(dict_gene[key], 'hasRNATranscript', w)
    writeEntry2(dict_gene[key], 'ncbiProteinAccessionNumber', w, True)
    writeEntry2(dict_gene[key], 'description', w, True)
    w.write('typeOf: schema:Gene\n')
    w.write('\n')
w.close()
print('Number of Gene instances: ' + str(count))

Number of Gene instances: 58683


In [6]:
# write the NCBI gene file to dict_gene_ncbi
dict_gene_ncbi = {}
NCBI_gene_file = species + '.gene_info'
r = open(NCBI_gene_file, 'r')
next(r)
for line in r:
    line = line.strip('\r\n').strip('\\').split('\t')
    if line[2] not in dict_gene:
        dict_gene_ncbi[line[2]] = {'inChromosome': 'chr' + line[6]}
        dict_gene_ncbi[line[2]]['description'] = line[8].strip('-')
    elif line[2] not in dict_gene_ncbi:
        dict_gene_ncbi[line[2]] = {}
    addToDict(dict_gene_ncbi[line[2]], 'ncbiTaxonID', line[0].strip('-'))
    addToDict(dict_gene_ncbi[line[2]], 'ncbiGeneID', line[1].strip('-'))
    addToDict(dict_gene_ncbi[line[2]], 'ncbiLocusTag', line[3].strip('-'))
    addToDict(dict_gene_ncbi[line[2]], 'mapLocation', line[7].strip('-'))
    addToDict(dict_gene_ncbi[line[2]], 'typeOfGene', line[9].strip('-'))
    addToDict(dict_gene_ncbi[line[2]], 'fullName', line[11].strip('-'))
    addToDict(dict_gene_ncbi[line[2]], 'modificationDate', line[14].strip('-'))
    addToDict(dict_gene_ncbi[line[2]], 'nomenclatureStatus', line[12])
    HGNC = ''
    mim = ''
    ensembl = ''
    otherNames = line[2].split('|')
    for name in otherNames:
      # split up the alternative names to correspond to the correct property
        n = name.split(':')
        if name.startswith('HGNC'):
            HGNC = n[-1]
        elif name.startswith('mim'):
            MIM = n[-1]
        elif name.startswith('Ensembl'):
            ensembl = n[-1]
        elif name == line[2]:
            continue
        else:
            print(name)
        addToDict(dict_gene_ncbi[line[2]], 'hgncID', HGNC)
        addToDict(dict_gene_ncbi[line[2]], 'mimID', mim)
        addToDict(dict_gene_ncbi[line[2]], 'ensemblID', ensembl)
r.close()

In [7]:
# write the gene nodes to a file in mcf format
gene_ncbi_output_file = assembly + '_gene_ncbi.mcf'
gene_mcf_file = os.path.join(path, 'ncbi_Gene_mcfs', gene_ncbi_output_file)
w = open(gene_mcf_file, 'w')
count = 0
for key in dict_gene_ncbi:
    count += 1
    dcid = 'bio/' + assembly + '_' + key
    w.write('Node: dcid:' + dcid + '\n')
    w.write('name: "' + key + '"\n')
    w.write('ofSpecies: dcid:bio/' + species_abv + '\n')
    w.write('inGenomeAssembly: dcid:bio/' + assembly + '\n')
    if 'inChromosome' in dict_gene_ncbi[key]:
        w.write('inChromosome: dcid:bio/' + assembly + '_' + dict_gene_ncbi[key]['inChromosome'] + '\n')
    writeEntry2(dict_gene_ncbi[key], 'mapLocation', w, True)
    writeTypeOfGene(dict_gene_ncbi[key]['typeOfGene'], w)
    w.write('geneSymbol: "' + key + '"\n')
    writeEntry2(dict_gene_ncbi[key], 'ncbiTaxonID', w, False)
    writeEntry2(dict_gene_ncbi[key], 'ncbiGeneID', w, False)
    writeEntry2(dict_gene_ncbi[key], 'ncbiLocusTag', w, True)
    writeEntry2(dict_gene_ncbi[key], 'mimID', w, True)
    writeEntry2(dict_gene_ncbi[key], 'hgncID', w, True)
    writeEntry2(dict_gene_ncbi[key], 'ensemblID', w, True)
    writeEntry2(dict_gene_ncbi[key], 'fullName', w, True)
    writeNomenclatureStatus(dict_gene_ncbi[key]['nomenclatureStatus'], w)
    writeEntry2(dict_gene_ncbi[key], 'modificationDate', w, False)
    writeEntry2(dict_gene_ncbi[key], 'description', w, True)
    w.write('typeOf: schema:Gene\n')
    w.write('\n')
w.close()
print('Number of Gene instances: ' + str(count))

Number of Gene instances: 61559
